In [1]:
from marvin.tools import Maps
import marvin
import numpy as np
import matplotlib.pyplot as plt

[WARNING]: cannot initiate Sentry error reporting: [Errno 25] Inappropriate ioctl for device. (UserWarning)


[INFO]: No release version set. Setting default to DR17


[WARNING]: path /home/neap/sas/dr17/manga/spectro/analysis/v3_1_1/3.1.0/dapall-v3_1_1-3.1.0.fits cannot be found. Setting dapall to None. (MarvinUserWarning)


In [2]:
def mask_PSB(hvalue, dvalue, divar, snr): 
    psb = np.zeros(hvalue.shape, dtype=bool)

    var = 1 / np.sqrt(divar)
    psb[np.logical_and(np.logical_and(hvalue < 3, (dvalue - var) > 4), snr > 3)] = True

    return psb

In [3]:
def get_histogram(galaxy_id):
    np.seterr(divide='ignore')
    galaxy = marvin.tools.Maps(galaxy_id, bintype='HYB10')
    alpha = galaxy.emline_gew_ha_6564
    delta = galaxy.specindex_hdeltaagalaxy
    snr = galaxy.spx_snr
    snr_value = getattr(snr, 'value', None)
    value = getattr(alpha, 'value', None)
    dvalue = getattr(delta, 'value', None)
    divar = getattr(delta, 'ivar', None)
    maskpsb = mask_PSB(value, dvalue, divar, snr_value)

In [4]:
def get_total_spaxels(dapmap):
    value = getattr(dapmap, 'value', None)
    ivar = getattr(dapmap, 'ivar', None)
    low_snr = mask_low_snr(value, ivar, 1)
    nocov = dapmap.pixmask.get_mask('NOCOV')
    good_spax = np.ma.array(value, mask=np.logical_or.reduce((nocov, low_snr)))
    valid_counts = good_spax.count()
    return valid_counts

def mask_low_snr(value, ivar, snr_min):

    low_snr = np.zeros(value.shape, dtype=bool)

    if (ivar is not None) and (not np.all(np.isnan(ivar))):
        low_snr = (ivar == 0.)

        if snr_min is not None:
            low_snr[np.abs(value * np.sqrt(ivar)) < snr_min] = True

    return low_snr

In [5]:
def get_histogram(galaxy_id):
    np.seterr(divide='ignore')
    galaxy = marvin.tools.Maps(galaxy_id, bintype='HYB10')
    alpha = galaxy.emline_gew_ha_6564
    delta = galaxy.specindex_hdeltaagalaxy
    snr = galaxy.spx_snr
    snr_value = getattr(snr, 'value', None)
    
    value = getattr(alpha, 'value', None)
    dvalue = getattr(delta, 'value', None)
    divar = getattr(delta, 'ivar', None)
    
    maskpsb = mask_PSB(value, dvalue, divar, snr_value)
    total_spax = get_total_spaxels(alpha)
    #print(np.count_nonzero(maskpsb))

    masks, fig, axes = galaxy.get_bpt(use_oi = False, show_plot=False)